# overall goals
- determine if we should use api or csv
- make code that downloads, updates, cleans, and loads the data
- make the downloads dependent on a "current year" variable
- figure out how to feed specific date ranges to the api if it's spitting back date-limited data
- make it dynamic enough to work in following years

In [1]:
import pandas as pd
import requests
import json
import datetime


In [ ]:

# #### in this cell we get define this year and last year
# current_year = datetime.datetime.now().year
# last_year = current_year - 1
# print(current_year)
# print(last_year)

In [ ]:
# ####  then use the api_year variable to determine which year we'll be calling
# #### using this variable to set which year's api we'll be calling
# api_year = last_year

# #### api call for the current year of data
# url = f'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Police_Incidents_{api_year}/FeatureServer/0/query?where=1%3D1&outFields=reportedDateTime,offense,description,centerLong,centerLat&outSR=4326&f=json'

In [ ]:
# ### make api request
# response = requests.get(url).json()

In [ ]:
# ### prints the response from the api call
# print(json.dumps(response,indent=4,sort_keys=True))

In [ ]:

# ### write response to json file
# with open(f'../resources/Police_Incidents_{api_year}.json','w+') as json_file:
#     json.dump(response,json_file)

In [ ]:
# #### open up and load json file as a variable
# with open(f'../resources/Police_Incidents_{api_year}.json') as json_file:
#     myjson = json.load(json_file)

In [ ]:
# # response['features'][0]
# len(response['features'])
# response['features']

In [ ]:
# ###### answering some basic questions in this cell: Whats the oldest and youngest 

# ##### clunkly way of getting year and month into a dataframe
# month_list = []
# year_list = []
# datetime_list = []

# for x in myjson['features']:

#     #### get the unix timestamp for this entry's time, divide by 1000 to remove milliseconds and convert it to date time
#     # print(datetime.datetime.fromtimestamp(x['attributes']['reportedDateTime']/1000))
#     feature_date = datetime.datetime.fromtimestamp(x['attributes']['reportedDateTime']/1000)
#     datetime_list.append(feature_date)
#     month_list.append(feature_date.month)
#     year_list.append(feature_date.year)

# ym_df = pd.DataFrame({'month':month_list,'year':year_list,'dt':datetime_list})

# del datetime_list
# del month_list
# del year_list


# ### find the minimum and maximum dates 
# ym_df['dt'].min()
# ym_df['dt'].max()


# print('oldest date')
# print(ym_df['dt'].min())
# print('--------------------------------------')
# print('newest date')
# print(ym_df['dt'].max())
# print('--------------------------------------')
# print('--------------------------------------')
# print('--------------------------------------')

# # ## groupby year and month and do an aggregate count to see the distribution of the incidents

# ym_df.groupby(['year','month']).count()




In [2]:
#### in this cell we get define this year and last year
current_year = datetime.datetime.now().year
last_year = current_year - 1
print(current_year)
print(last_year)

2021
2020


In [6]:
#### make a cell that makes 2 api calls, for this year and the prior, and joins the data from each call into a single variable


### calls api and returns the features for a given year
def callAndStore(year):
    url = f'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Police_Incidents_{year}/FeatureServer/0/query?where=1%3D1&outFields=reportedDateTime,offense,description,centerLong,centerLat&outSR=4326&f=json'
    ### make api request
    response = requests.get(url).json()

    features = [x for x in response['features']]

    # ### write response to json file
    # with open(f'../resources/Police_Incidents_{year}.json','w+') as json_file:
    #     json.dump(response,json_file)

    return features

feature_list=[]


######### option 2 that returns the date and time as seperate entries
##### for all features in the current and prior year.....
for feature in (callAndStore(current_year)+callAndStore(last_year)):
    clean_feature={
        'date':datetime.datetime.fromtimestamp(feature['attributes']['reportedDateTime']/1000).strftime("%m/%d/%Y"),
        'time':datetime.datetime.fromtimestamp(feature['attributes']['reportedDateTime']/1000).strftime("%H:%M:%S"),
        'centerLong': feature['attributes']['centerLong'],
        'centerLat':feature['attributes']['centerLat'],
        'description':feature['attributes']['description'].strip()
        }
    feature_list.append(clean_feature)



######### option 2 that returns the entire date time string
# ##### for all features in the current and prior year.....
# for feature in (callAndStore(current_year)+callAndStore(last_year)):
#     # print(datetime.datetime.fromtimestamp(feature['attributes']['reportedDateTime']/1000))
#     # print(type(datetime.datetime.fromtimestamp(feature['attributes']['reportedDateTime']/1000)))
#     clean_feature={
#         'reportedDateTime':datetime.datetime.fromtimestamp(feature['attributes']['reportedDateTime']/1000).strftime("%m/%d/%Y %H:%M:%S"),
#         'centerLong': feature['attributes']['centerLong'],
#         'centerLat':feature['attributes']['centerLat'],
#         'description':feature['attributes']['description'].strip()
#         }
#     feature_list.append(clean_feature)


### write list of feature to json file
with open(f'../resources/Police_Incidents_{last_year}_to_{current_year}.json','w+') as json_file:
    json.dump(feature_list,json_file)




In [7]:
feature_list[0]

{'reportedDateTime': '01/04/2021 01:41:00',
 'centerLong': -93.28019274,
 'centerLat': 44.98504707,
 'description': 'AUTOMOBILE THEFT'}

In [ ]:

# myjson['objectIdFieldName']
# myjson['uniqueIdField']
# myjson['globalIdFieldName']
# myjson['geometryType']
# myjson['spatialReference']
# myjson['fields']
# myjson['features']

#####-----------
#####-----------
#####-----------